In [1]:
import numpy as np
import pickle
from scipy.interpolate import interp1d
from scipy.io import wavfile
import librosa 

In [2]:
with open('trajectories.pkl', 'rb') as f:
    trajectories = pickle.load(f)[0]

In [3]:
body1_positions = trajectories[0]
body2_positions = trajectories[1]
body3_positions = trajectories[2]

In [4]:
def cartesian_to_polar(x, y):
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    return r, theta

In [5]:
body1_polar = np.array([cartesian_to_polar(x, y) for x, y in body1_positions])
body2_polar = np.array([cartesian_to_polar(x, y) for x, y in body2_positions])
body3_polar = np.array([cartesian_to_polar(x, y) for x, y in body3_positions])

In [6]:
audio_sample_rate = 44100  # Sample rate in Hz
sim_sample_rate = 1000 # Sample rate of the simulation in Hz
duration = len(body1_polar) / sim_sample_rate # Duration of the audio in seconds
t_audio = np.linspace(0, duration, int(duration * audio_sample_rate), endpoint=False)

In [7]:
def polar_to_audio(r, theta, t_audio):
    # Keep track of the number of rotations
    rotations = np.cumsum(np.abs(np.diff(theta)) > np.pi)
    rotations = np.insert(rotations, 0, 0)

    # Adjust the angle based on the number of rotations
    adjusted_theta = theta + rotations * 2 * np.pi

    # Create interpolation functions for r and adjusted_theta
    r_interp = interp1d(np.linspace(0, 1, len(r)), r, kind='cubic')
    theta_interp = interp1d(np.linspace(0, 1, len(adjusted_theta)), adjusted_theta, kind='cubic')

    # Interpolate r and adjusted_theta at the audio timestamps
    r_audio = r_interp(np.linspace(0, 1, len(t_audio)))
    theta_audio = theta_interp(np.linspace(0, 1, len(t_audio)))

    frequency = 440 * 2 ** (theta_audio / (2 * np.pi))  # Convert adjusted angle to frequency
    amplitude = r_audio / np.max(r_audio)  # Normalize the distance to amplitude (0 to 1)
    audio = amplitude * np.sin(2 * np.pi * frequency * t_audio)
    return audio

audio1 = polar_to_audio(body1_polar[:, 0], body1_polar[:, 1], t_audio)
audio2 = polar_to_audio(body2_polar[:, 0], body2_polar[:, 1], t_audio)
audio3 = polar_to_audio(body3_polar[:, 0], body3_polar[:, 1], t_audio)

In [8]:
# audio = np.vstack((audio1, audio2, audio3)).T
audio_mixed = audio1 + audio2 + audio3
audio_mixed = audio_mixed / np.max(np.abs(audio_mixed))

In [9]:
audio_fast = librosa.effects.time_stretch(audio_mixed, rate=4)

In [10]:
# Write the mixed audio to a WAV file
wavfile.write("3body_audio_fast.wav", 44100, audio_fast)